## This python notebook outlines a method to generate tweets 
## The tweets provide airbnb recommondations and top food choices nearby given an inputed  location/zipcode. 
### It automatically tweets/replies when the user runs the portion of code

## Getting started with the yelp api

In [1]:
import requests
import pandas as pd
import re
import random

In [2]:
client_id = #INSERT CLIENT ID FOR YELP FUSION API HERE (STRING)***********************************
client_secret = #INSERT CLIENT SECRET FOR YELP FUSION API HERE ***********************************

In [3]:
#pip install yelpapi in the cmd line
from yelpapi import YelpAPI
yelp_api = YelpAPI(client_id, client_secret)
search_results = yelp_api.search_query(location = "00165") #practice using the search query

In [56]:
search_results.get("total")
#1497 businesses in zip code 00165

1497

In [5]:
type(brunch.at[2,'zipcode']) #zipcode is type string, for future reference

str

In [ ]:
#Visualize json-like (complicated dictionary-list combination) output to analyze later
breakfast = yelp_api.search_query(categories = "breakfast_brunch,cafes", latitude = 41.876854, longitude = 12.539091, radius = 1000, sort_by= 'rating', price = "1,2", limit = 50)
breakfast.get("businesses")[3]

## Load Airbnb Data
### Download csv from http://insideairbnb.com/get-the-data.html
#### (Or scrape it yourself)

In [5]:
#Read the data in from your local drive
listings = pd.read_csv("C:/Users/Student/Desktop/listings.csv", dtype={"license" : str, "zipcode" : str })

In [ ]:
#get list of column values to understand what we have
list(listings.columns.values)

In [67]:
#Select relevant columns
df = listings[["host_name","listing_url","room_type","review_scores_rating","review_scores_location","latitude","longitude","beds","price","accommodates","number_of_reviews"]]

In [68]:
#Use regular expressions so that the price column can be used and manipulated
df.price = (df['price'].replace( '[\$,)]','', regex=True ).astype(float))
#check
df.tail()
type(df.at[120,'price'])

C:\Users\Student\Anaconda3\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,host_name,listing_url,room_type,review_scores_rating,review_scores_location,latitude,longitude,beds,price,accommodates,number_of_reviews
25270,Alessia,https://www.airbnb.com/rooms/18144886,Private room,100.0,10.0,41.823673,12.635800,4.0,55.0,5,1
25271,Loredana,https://www.airbnb.com/rooms/18149478,Entire home/apt,NaN,NaN,41.802475,12.595191,1.0,48.0,2,0
25272,Katia,https://www.airbnb.com/rooms/18326174,Entire home/apt,NaN,NaN,41.734215,12.259200,3.0,89.0,5,0
25273,Andrea,https://www.airbnb.com/rooms/16872329,Entire home/apt,NaN,NaN,41.909047,12.262159,2.0,45.0,1,0
25274,Silvia,https://www.airbnb.com/rooms/18054968,Entire home/apt,NaN,NaN,41.825853,12.727577,3.0,36.0,4,0


In [7]:
#check type in other columns
type(df.at[120,'accommodates'])

numpy.int64

In [69]:
#Filter the airbnb search into a smaller list since Rome is so large
#Adjust price range and rating as preferred
df_filtered = df.query('price<120').query('review_scores_rating>95').query('review_scores_location>=8')
#reset index 
df_filtered = df_filtered.reset_index()
df_filtered.tail()

In [71]:
#These "fluff" words allow tweets to be Unique
opening_line = ['Accommodation for', '#Suggestion for','Trip for','Itinerary for','Getaway for',"Rome! For"]
next_line =['Enjoy', "Head to", "We recommend", "", "Test",  "Try","Test out", "Eat at"]
drink_line = ['Grab a drink at', 'Sit down and stay awhile at', "Experience the nightlife at"]
hashtag = ["#RVaction", "#Rome", "#ROMEing", "#Travel","ROMEantic"]

## Generate tweets using yelp, airbnb filtered data
### Replies generated to provide reference to booking with airbnb host

In [ ]:
running = True

In [93]:
tweets = []
reply_tweets = []

In [94]:
#Follow prompts, everything is randomized, ensures tweets fit 140 character limit
while running == True:
    begin = input("Do you want to generate a tweet? (y/n)")
    
    if (begin == 'y'):
        rand = random.randint(0,len(df_filtered.index) - 1) 
        host_name = df_filtered.at[rand,'host_name']
        price = df_filtered.at[rand,'price']
        lat_airbnb = df_filtered.at[rand,'latitude']
        long_airbnb = df_filtered.at[rand,'longitude']
        accommodates = df_filtered.at[rand,'accommodates']
        rand_line = random.randint(0, len(opening_line) - 1)
        next_rand = random.randint(0, len(next_line) - 1)
        
        breakfast = (yelp_api.search_query(term = "breakfast,brunch", latitude = lat_airbnb, longitude = long_airbnb, radius = 1000, sort_by= 'rating', price = "1,2", limit = 50))
        dinner = yelp_api.search_query(term = "dinner", latitude = lat_airbnb, longitude = long_airbnb, radius = 1000, sort_by= 'rating', price = "1,2", limit = 50)
        drinks = yelp_api.search_query(categories = "bars", latitude = lat_airbnb, longitude = long_airbnb, radius = 1000, sort_by= 'rating', price = "1,2", limit = 50)
        if (breakfast.get("total") < 2 or dinner.get("total") < 2 or drinks.get("total") < 2):
            print("Airbnb didn't produce enough results. Trying again...")
        else:
            
            rand_brunch = random.randint(0, len(breakfast.get("businesses")) - 1)
            breakfast_place = ((breakfast.get("businesses"))[rand_brunch]).get("name")
            
            rand_din = random.randint(0, len(dinner.get("businesses")) -1)
            dinner_place = ((dinner.get("businesses"))[rand_din]).get("name")
            
            rand_drank = random.randint(0, len(drinks.get("businesses")) -1)
            drinks_place = ((drinks.get("businesses"))[rand_drank]).get("name")
            rand_hashtag = random.randint(0, len(hashtag) -1)
            new_tweet = opening_line[rand_line] + " " + str(accommodates) + " at " + host_name + "'s place. "  + next_line[next_rand] + " " + breakfast_place + " for breakfast, " + dinner_place + " for dinner and " + drinks_place + " for drinks! " + hashtag[rand_hashtag]
            reply_url = df_filtered.at[rand,'listing_url']
            reply_tweet = "Find out about " + host_name + "'s airbnb at: " + reply_url
            
            if len(new_tweet) <= 140:
                print("New tweet generated.")
                tweets.append(new_tweet)
                reply_tweets.append(reply_tweet)
            else:
                print("Tweet too long :(")
                
    else:
        print("Tweet generation stopped. You generated these tweets:")
        print(tweets)
        running == False
        break


Do you want to generate a tweet? (y/n)y
New tweet generated.
Do you want to generate a tweet? (y/n)n
Tweet generation stopped. You generated these tweets:
["Getaway for 5 at Gabriella's place. Enjoy Agorà for breakfast, Asahi for dinner and Il Serpente for drinks! #Rome"]


In [96]:
#breakfast = (yelp_api.search_query(term = "breakfast,brunch", latitude = lat_airbnb, longitude = long_airbnb, radius = 1000, sort_by= 'rating', price = "1,2", limit = 50))
#breakfast_place = ((breakfast.get("businesses"))[0]).get("name")
#print(len(breakfast.get('businesses')))
#print(breakfast.get('total'))
#print(breakfast_place)
for tweet in reply_tweets:
    print(tweet)

Find out about Gabriella's airbnb at: https://www.airbnb.com/rooms/5639167


# Using twitter api to send tweets

In [ ]:
from twython import Twython
import webbrowser as wb
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

In [ ]:
app_key = #INSERT YOUR APP KEY HERE FROM TWITTER DEVELOPER 
app_secret = #INSERT YOUR APP SECRET HERE

In [99]:
twitter = Twython(app_key, app_secret)
auth = twitter.get_authentication_tokens(callback_url='https://google.com')

Oauth_Token = auth['oauth_token']
Oauth_Token_Secret = auth['oauth_token_secret']
#install webdrive for chrome (or other browser), download to desktop
driver = webdriver.Chrome(executable_path = "C:/Users/Student/Desktop/chromedriver")

In [ ]:
start = input("Welcome to Twitter authorization. Would you like to continue or terminate? (y / n)")
if (start == "y"):
    driver.get(auth['auth_url'])
    try:
        driver.find_element_by_name('session[username_or_email]').send_keys()#Insert Username as argument
        driver.find_element_by_name('session[password]').send_keys()#Insert Passoword in parentheses
        driver.find_element_by_id('allow').click()
    except NoSuchElementException:
        print("Not the login page")
    time.sleep(5)
    ex = re.compile('(?<=oauth_verifier=)[a-zA-Z0-9]+')
    verifier = ex.findall(driver.current_url)
    t = Twython(app_key, app_secret, Oauth_Token, Oauth_Token_Secret)
    final_step = t.get_authorized_tokens(verifier)
    tw = Twython(app_key, app_secret, final_step['oauth_token'], final_step['oauth_token_secret'])
    print(final_step)
    tw.update_status(status = tweets[0]) #change index 0 to the specific tweet you want or change to a loop to tweet all generated tweets.
    time.sleep(5)
    recent_tweet = tw.get_user_timeline(user_id = IDHERE, count = 1, include_rts = False)
    status_id = recent_tweet[0]['id']
    tw.update_status(status = reply_tweets[0], in_reply_to_status_id = status_id) #change index 0 of reply_tweets to same as tweets[INDEX]
    
    timeLine = input("See timeline? (y / n)")
    if (timeLine == "y"):
        print(tw.get_home_timeline())
    else:
        print("Program Stop")
        break
else:
    print("Program Stop")
    break